# Projeto Integrador 6

## Predição de diabetes com modelos de IA

## Documentação dataset -> https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators

# DADOS BRUTOS

## Import das Bibliotecas e Pacotes

In [1]:
!pip install optuna
!pip install ucimlrepo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import optuna
from plotly.io import show
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, FunctionTransformer, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.compose import ColumnTransformer
from ucimlrepo import fetch_ucirepo
from functools import partial

## Import do Dataset

In [3]:
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

df = cdc_diabetes_health_indicators.data.features
aux = cdc_diabetes_health_indicators.data.targets

df["diabetes"] = aux["Diabetes_binary"]


print(cdc_diabetes_health_indicators.metadata)

print(cdc_diabetes_health_indicators.variables)

{'uci_id': 891, 'name': 'CDC Diabetes Health Indicators', 'repository_url': 'https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators', 'data_url': 'https://archive.ics.uci.edu/static/public/891/data.csv', 'abstract': 'The Diabetes Health Indicators Dataset contains healthcare statistics and lifestyle survey information about people in general along with their diagnosis of diabetes. The 35 features consist of some demographics, lab test results, and answers to survey questions for each patient. The target variable for classification is whether a patient has diabetes, is pre-diabetic, or healthy. ', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Multivariate'], 'num_instances': 253680, 'num_features': 21, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Sex', 'Age', 'Education Level', 'Income'], 'target_col': ['Diabetes_binary'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_

<ipython-input-3-96fc3f93fe39>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [4]:
df.sample(20)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,diabetes
249578,1,1,1,23,0,0,0,0,1,1,...,0,3,0,3,0,0,8,2,5,0
198721,0,0,1,28,1,0,0,1,1,1,...,0,2,0,0,0,1,11,5,8,0
180845,1,1,1,27,0,0,0,1,0,1,...,0,3,0,1,1,0,11,4,6,0
19217,1,1,1,30,0,0,0,1,0,0,...,1,4,0,0,0,1,2,4,6,0
141137,1,0,1,38,1,1,0,0,1,1,...,0,4,0,15,0,0,8,5,5,0
16964,0,0,1,26,0,0,0,1,0,1,...,0,1,0,0,0,1,3,2,1,0
173192,0,1,1,23,0,0,0,1,1,1,...,0,2,0,2,0,0,5,5,8,0
31378,1,0,1,46,1,0,0,1,1,1,...,0,4,30,3,0,0,8,4,5,1
248384,1,1,1,31,0,0,0,0,1,1,...,0,3,0,0,0,0,8,6,8,0
27488,0,1,1,29,0,0,0,1,0,0,...,0,3,0,0,0,1,8,5,5,0


# Separação de conjuntos estratificados de aprendizado, teste e validação

In [43]:
X = df.drop(columns=["diabetes"])
y = df["diabetes"]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)

categorical_columns = ['GenHlth', 'Age', 'Education', 'Income']
numerical_columns = ['BMI', 'MentHlth', 'PhysHlth']

print(y_train.value_counts())
print(y_test.value_counts())

x_train_split, x_validation, y_train_split, y_validation = train_test_split(x_train, y_train, test_size=0.2, random_state=10, stratify=y_train)

print(y_train_split.value_counts())
print(y_validation.value_counts())

diabetes
0    174667
1     28277
Name: count, dtype: int64
diabetes
0    43667
1     7069
Name: count, dtype: int64
diabetes
0    139733
1     22622
Name: count, dtype: int64
diabetes
0    34934
1     5655
Name: count, dtype: int64


# PIPELINE

## KNN

### Pré-processamento + Seleção de Hiperparâmetros

In [44]:
def remove_outliers_iqr(x, y, columns):
    condition = pd.Series(True, index=x.index)

    for column in columns:
        Q1 = x[column].quantile(0.25)
        Q3 = x[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Atualiza a condição para incluir a coluna atual
        condition &= (x[column] >= lower_bound) & (x[column] <= upper_bound)

    # Aplica a condição para filtrar x e y
    x_filtered = x[condition]
    y_filtered = y[condition]

    return x_filtered, y_filtered



In [45]:
x_train_split.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
146728,0,1,1,27,1,0,0,1,1,1,...,1,0,2,0,0,0,0,8,5,8
77325,0,0,1,32,0,0,0,1,1,1,...,1,0,1,3,0,0,0,4,6,7
24237,0,1,1,32,0,0,0,1,0,1,...,1,0,3,0,0,0,1,9,5,7
29668,0,1,1,34,1,0,0,1,1,1,...,1,0,3,1,2,0,0,11,6,7
122763,1,1,1,24,0,0,1,1,1,1,...,1,0,4,0,30,0,0,13,4,3


In [46]:
# x_train_split, y_train_split = remove_outliers_iqr(x_train_split, y_train_split, columns=['BMI', 'PhysHlth', 'MentHlth'])

preprocessor_knn = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ])



In [47]:
preprocessor_knn.fit_transform(x_train_split).todense().shape

(162355, 35)

In [48]:
def objective(trial):
    pipeline = Pipeline([
        #('outlier_remover', outlier_remover),
        # ("train_validation_split", FunctionTransformer(train_validation_split)),
        ('preprocessor', preprocessor_knn),
        ('knn', KNeighborsClassifier())
    ])
    k = trial.suggest_int('k', 2, 15) # alteração de 25 para 15
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])
    pipeline.set_params(knn__n_neighbors=k, knn__weights=weights, knn__metric=metric)
    pipeline.fit(x_train_split, y_train_split)

    y_pred = pipeline.predict(x_validation)
    score = f1_score(y_validation, y_pred)
    return score


#objective_func = partial(objective, pipeline=pipe_knn)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10, n_jobs=-1) # incrementação do parâmetro n_jobs=-1
# study.optimize(objective_func, n_trials=100)

study.best_params

[I 2024-10-30 23:23:01,569] A new study created in memory with name: no-name-99268a21-0698-4893-a6af-6ca1ad4f53cc
[I 2024-10-30 23:31:14,117] Trial 0 finished with value: 0.12019009658132761 and parameters: {'k': 14, 'weights': 'uniform', 'metric': 'minkowski'}. Best is trial 0 with value: 0.12019009658132761.
[I 2024-10-30 23:31:16,040] Trial 1 finished with value: 0.12441207707479897 and parameters: {'k': 12, 'weights': 'uniform', 'metric': 'minkowski'}. Best is trial 1 with value: 0.12441207707479897.
[I 2024-10-30 23:38:28,093] Trial 2 finished with value: 0.21754645010458964 and parameters: {'k': 9, 'weights': 'distance', 'metric': 'minkowski'}. Best is trial 2 with value: 0.21754645010458964.
[I 2024-10-30 23:43:16,561] Trial 3 finished with value: 0.25301459699598056 and parameters: {'k': 2, 'weights': 'distance', 'metric': 'manhattan'}. Best is trial 3 with value: 0.25301459699598056.
[I 2024-10-30 23:50:27,224] Trial 4 finished with value: 0.1500280426247897 and parameters: {'

{'k': 2, 'weights': 'distance', 'metric': 'manhattan'}

In [ ]:
with open('diabetes_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Modelo salvo com sucesso em 'diabetes_model.pkl'.")
